In [1]:
#from anytree import Node, RenderTree, NodeMixin, search, PreOrderIter
#from scipy.stats import chi2_contingency
import pandas as pd
import numpy as np

#from optbinning import MulticlassOptimalBinning #http://gnpalencia.org/optbinning/binning_multiclass.html
#import rpy2.robjects as robjects
#from rpy2.robjects import pandas2ri
#from rpy2.robjects.conversion import localconverter
#import rpy2.rinterface as rinterface
#from rpy2.robjects.packages import importr, data
#from itertools import combinations
#
#from rpy2.robjects import pandas2ri

from cart import *


In [2]:
'''
train = pd.read_csv("Carseats_train.csv")
test = pd.read_csv("Carseats_test.csv")
print(train.shape)

high_train = []

for i in range(train.shape[0]):
    if train["Sales"][i] > 8:
        high_train.append("Yes")
    else:
        high_train.append("No")

train["High"] = high_train
del train["Sales"]
del train["Unnamed: 0"]

indici = np.arange(0, len(high_train))

high_test = []
for i in range(test.shape[0]):
    if test["Sales"][i] > 8:
        high_test.append("Yes")
    else:
        high_test.append("No")

test["High"] = high_test
del test["Sales"]
del test["Unnamed: 0"]

feature_var = "High"
num_var = ["CompPrice", "Income", "Advertising", "Population", "Price", "Age", "Education"]
cat_var = ["ShelveLoc", "Urban", "US"]

display(train.head())


y = train["High"].copy()
del train["High"]
features = train[num_var]
features_names = num_var
n_features = train[cat_var]
n_features_names = cat_var
'''

'\ntrain = pd.read_csv("Carseats_train.csv")\ntest = pd.read_csv("Carseats_test.csv")\nprint(train.shape)\n\nhigh_train = []\n\nfor i in range(train.shape[0]):\n    if train["Sales"][i] > 8:\n        high_train.append("Yes")\n    else:\n        high_train.append("No")\n\ntrain["High"] = high_train\ndel train["Sales"]\ndel train["Unnamed: 0"]\n\nindici = np.arange(0, len(high_train))\n\nhigh_test = []\nfor i in range(test.shape[0]):\n    if test["Sales"][i] > 8:\n        high_test.append("Yes")\n    else:\n        high_test.append("No")\n\ntest["High"] = high_test\ndel test["Sales"]\ndel test["Unnamed: 0"]\n\nfeature_var = "High"\nnum_var = ["CompPrice", "Income", "Advertising", "Population", "Price", "Age", "Education"]\ncat_var = ["ShelveLoc", "Urban", "US"]\n\ndisplay(train.head())\n\n\ny = train["High"].copy()\ndel train["High"]\nfeatures = train[num_var]\nfeatures_names = num_var\nn_features = train[cat_var]\nn_features_names = cat_var\n'

In [3]:
'''
my_tree = MyNodeClass('n1', indici) 
cart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "gini",problem="classifier",method = "CART",max_level = 10, min_cases_parent= 10,min_cases_child= 5) 
cart.growing_tree(my_tree)
cart.print_tree() 
'''

'\nmy_tree = MyNodeClass(\'n1\', indici) \ncart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "gini",problem="classifier",method = "CART",max_level = 10, min_cases_parent= 10,min_cases_child= 5) \ncart.growing_tree(my_tree)\ncart.print_tree() \n'

In [4]:
#may need to bin numerical vars
#i think only works for categorical vars or binned vars 

In [5]:
pd.options.display.max_columns = None
dataframe = pd.read_table('lpmc.dat')
print(dataframe.shape)
dataframe.head()

(81086, 32)


,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,travel_year,travel_month,travel_date,day_of_week,start_time,age,female,driving_license,car_ownership,distance,dur_walking,dur_cycling,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,0,0,0,0,4,3,1,1,1.0,1,2012,4,1,7,10.000000,56,1,1,2,778,0.218056,0.092222,0.134444,0.0,0.016667,0.0,0,0.052222,1.5,0.14,0.0,0.111702
1,1,0,0,1,4,3,1,1,1.0,1,2012,4,1,7,13.000000,56,1,1,2,778,0.188889,0.061944,0.109444,0.0,0.055556,0.0,0,0.059444,1.5,0.15,0.0,0.112150
2,2,0,0,2,4,3,1,1,1.0,1,2012,4,1,7,17.000000,56,1,1,2,4562,1.037778,0.355833,0.203056,0.0,0.210278,0.0,0,0.236667,1.5,0.79,0.0,0.203052
3,3,0,0,3,4,3,1,1,1.0,1,2012,4,1,7,17.350000,56,1,1,2,4562,1.039444,0.384444,0.205556,0.0,0.258611,0.0,0,0.233333,1.5,0.78,0.0,0.160714
4,4,0,1,2,4,3,1,4,1.0,1,2012,4,1,7,19.583333,54,0,1,2,4562,1.037778,0.355833,0.203056,0.0,0.189444,0.0,0,0.229167,1.5,0.78,0.0,0.130909


In [6]:
#binning age

groupDict={'0-18':0,'19-30':0,'31-65':0, '66+': 0}

for i in dataframe['age']:
    if i>=0 and i<=10:
        groupDict['0-18']+=1
    elif i>=19 and i<=30:
        groupDict['19-30']+=1
    elif i>=31 and i<=65:
        groupDict['31-65']+=1
    else:
        groupDict['66+']+=1

dataframe.loc[dataframe['age']<=18, 'age_group'] = '0-18'
dataframe.loc[dataframe['age'].between(19,30), 'age_group'] = '19-30'
dataframe.loc[dataframe['age'].between(31,65), 'age_group'] = '31-65'
dataframe.loc[dataframe['age']>65, 'age_group'] = '66+'
dataframe.drop(['age'], axis= 1, inplace = True)


#binning distance

dataframe.loc[dataframe['distance']< 658.50, 'distance_group'] = 'distance01'
dataframe.loc[dataframe['distance'].between(658.50,861.49), 'distance_group'] = 'distance02'
dataframe.loc[dataframe['distance'].between(861.50,1214.49), 'distance_group'] = 'distance03'
dataframe.loc[dataframe['distance'].between(1214.50,1514.49 ), 'distance_group'] = 'distance04'
dataframe.loc[dataframe['distance'].between(1514.50, 2156.49), 'distance_group'] = 'distance05'
dataframe.loc[dataframe['distance'].between(2156.50, 2759.49), 'distance_group'] = 'distance06'
dataframe.loc[dataframe['distance'].between(2759.50, 4391.49), 'distance_group'] = 'distance07'
dataframe.loc[dataframe['distance'].between(4391.50, 5369.49), 'distance_group'] = 'distance08'
dataframe.loc[dataframe['distance'].between(5369.50, 10087.49), 'distance_group'] = 'distance09'
dataframe.loc[dataframe['distance'].between(10087.50, 14695.49), 'distance_group'] = 'distance10'
dataframe.loc[dataframe['distance']>=14695.50, 'distance_group'] = 'distance11'
dataframe.drop('distance', axis = 1, inplace = True)

#binning start time

dataframe.loc[dataframe['start_time']<8.091666, 'starttime_group'] = 'early_morning'
dataframe.loc[dataframe['start_time'].between(8.091666,10.00833320), 'starttime_group'] = 'morning'
dataframe.loc[dataframe['start_time'].between(10.008333211,18.50833320), 'starttime_group'] = 'afternoon'
dataframe.loc[dataframe['start_time']> 18.50833321, 'starttime_group'] = 'evening'
dataframe.drop('start_time', axis = 1, inplace = True)

#binning cost transit

dataframe.loc[dataframe['cost_transit']< 1.52499998, 'cost_group'] = 'low_cost'
dataframe.loc[dataframe['cost_transit'].between(1.52499998,3.01999997), 'cost_group'] = 'medium_cost'
dataframe.loc[dataframe['cost_transit']> 3.01999998, 'cost_group'] = 'high_cost'
dataframe.drop('cost_transit', axis = 1, inplace = True)

#binning cost_driving_Fuel

dataframe.loc[dataframe['cost_driving_fuel']< 0.165, 'costdrivingfuel_group'] = 'costdrivingfuelgroup1'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.165,0.204), 'costdrivingfuel_group'] = 'costdrivingfuelgroup2'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.205,0.244), 'costdrivingfuel_group'] = 'costdrivingfuelgroup3'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.245, 0.294), 'costdrivingfuel_group'] = 'costdrivingfuelgroup4'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.295, 0.354), 'costdrivingfuel_group'] = 'costdrivingfuelgroup5'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.355, 0.44499998), 'costdrivingfuel_group'] = 'costdrivingfuelgroup6'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.44499999, 0.83499997), 'costdrivingfuel_group'] = 'costdrivingfuelgroup7'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.83499998, 1.17499994), 'costdrivingfuel_group'] = 'costdrivingfuelgroup8'
dataframe.loc[dataframe['cost_driving_fuel'].between(1.17499995, 2.31499993), 'costdrivingfuel_group'] = 'costdrivingfuelgroup9'
dataframe.loc[dataframe['cost_driving_fuel']> 2.31499994, 'costdrivingfuel_group'] = 'costdrivingfuelgroup10'
dataframe.drop('cost_driving_fuel', axis = 1, inplace = True)

#binning driving traddic percent

dataframe.loc[dataframe['driving_traffic_percent']< 0.24422751, 'trafficperc_group'] = '0low_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.24422751,0.36449698), 'trafficperc_group'] = '1mediumlow_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.36449699,0.49620750), 'trafficperc_group'] = '2medium_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.49620751, 0.57161700), 'trafficperc_group'] = '3mediumhigh_traffic'
dataframe.loc[dataframe['driving_traffic_percent']> 0.57161701, 'trafficperc_group'] = '4high_traffic'
dataframe.drop('driving_traffic_percent', axis = 1, inplace = True)

#binning cost_driving_ccharge

dataframe.loc[dataframe['cost_driving_ccharge']< 5.25, 'drivingcharge_group'] = 'low_charge'
dataframe.loc[dataframe['cost_driving_ccharge']> 5.25, 'drivingcharge_group'] = 'high_charge'
dataframe.drop('cost_driving_ccharge', axis = 1, inplace = True)

In [7]:
y = dataframe[['travel_mode', 'travel_year']]
X = dataframe.drop(['travel_mode', 'survey_year'], axis = 1)

print(set(X["travel_year"]))

a = X.loc[X["travel_year"]==2015]
#creating test set 

X_test= X.query('travel_year == 2015') #creates a subset, like this: X.loc[X["travel_year"]==2015]
y_test = y.query('travel_year == 2015')

X_test = X_test.drop(['travel_year'], axis =1).copy()
y_test = y_test.drop(['travel_year'], axis =1).copy()

#create training set 

X_train_1 = X.query('travel_year == 2012')
y_train_1 = y.query('travel_year == 2012')
X_train_2 = X.query('travel_year == 2013')
y_train_2 = y.query('travel_year == 2013')
X_train_3 = X.query('travel_year == 2014')
y_train_3 = y.query('travel_year == 2014')

X_train = pd.concat([X_train_1, X_train_2, X_train_3])
y_train = pd.concat([y_train_1, y_train_2, y_train_3])

X_train = X_train.drop(['travel_year'], axis = 1)
y_train = y_train.drop(['travel_year'], axis = 1)


data_to_shuffle = pd.concat([X_train, y_train], axis = 1)
data = data_to_shuffle.sample(frac=1).reset_index(drop=True) #sampling returning all, and resetting index , no seed?
y_train = data['travel_mode']
X_train = data.drop(['travel_mode'], axis = 1)

#feature selection

to_drop = ['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_date', 'dur_pt_int',
          'dur_pt_rail', 'dur_pt_access', 'dur_pt_bus', 'dur_walking', 'dur_cycling', 'dur_driving']
print(f'Number of columns dropped: {len(to_drop)}')
X_train.drop(to_drop, axis = 1, inplace = True)

X_test.drop(to_drop, axis = 1, inplace = True)

predictors = X_test.columns
y = y_test #is the whole vector
print(predictors)

datatest = pd.concat([y_test, X_test], axis = 1).reset_index()
datatest.drop('index', axis = 1, inplace = True)
datatest.to_csv("data_test.csv", index = False)

datatraining = pd.concat([y_train, X_train], axis = 1).reset_index()
datatraining.drop('index', axis = 1, inplace = True)
datatraining.to_csv("data_training.csv", index = False)


{2012, 2013, 2014, 2015}
Number of columns dropped: 12
Index(['purpose', 'fueltype', 'faretype', 'bus_scale', 'travel_month',
       'day_of_week', 'female', 'driving_license', 'car_ownership',
       'pt_interchanges', 'age_group', 'distance_group', 'starttime_group',
       'cost_group', 'costdrivingfuel_group', 'trafficperc_group',
       'drivingcharge_group'],
      dtype='object')


In [8]:
display(y_train.head(10))
print(X_train.shape)
X_train.head(10)

0    4
1    1
2    3
3    4
4    1
5    4
6    1
7    4
8    3
9    3
Name: travel_mode, dtype: int64

(76103, 17)


,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group
0,1,1,1,1.0,11,2,0,1,1,1,31-65,distance08,afternoon,high_cost,costdrivingfuelgroup8,2medium_traffic,low_charge
1,3,6,1,1.0,8,1,1,0,0,0,31-65,distance03,afternoon,low_cost,costdrivingfuelgroup4,0low_traffic,low_charge
2,3,1,1,1.0,10,1,1,1,2,0,31-65,distance09,afternoon,low_cost,costdrivingfuelgroup8,0low_traffic,low_charge
3,5,1,5,0.0,1,3,1,1,2,0,66+,distance03,afternoon,low_cost,costdrivingfuelgroup4,2medium_traffic,low_charge
4,5,6,1,1.0,12,5,0,1,0,0,31-65,distance01,afternoon,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
5,1,1,1,1.0,11,2,1,1,2,0,31-65,distance06,early_morning,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
6,5,1,1,1.0,9,4,0,1,1,0,31-65,distance01,afternoon,low_cost,costdrivingfuelgroup1,4high_traffic,high_charge
7,3,2,1,1.0,2,7,1,1,1,0,31-65,distance03,afternoon,low_cost,costdrivingfuelgroup2,0low_traffic,low_charge
8,1,1,1,1.0,11,2,1,0,1,2,31-65,distance09,evening,high_cost,costdrivingfuelgroup9,1mediumlow_traffic,low_charge
9,3,6,5,0.0,8,6,0,0,0,0,0-18,distance07,afternoon,low_cost,costdrivingfuelgroup7,2medium_traffic,low_charge


In [9]:
X_train.reset_index(inplace = True, drop = True)
y_train.reset_index(inplace = True, drop = True)

X_train=X_train.applymap(str)
#y_train=y_train.astype(str)

n_features = X_train
n_features_names = list(X_train.columns)
indici = np.arange(0, len(y_train))
y = y_train





In [10]:
#need to bin num vars and have features, features_names empty

#reched tau at each new split


#not doing optbinning internally  maybe could

features = []
features_names = []


#features, features_names, n_features, n_features_names, y

negative node prop gain? maybe ok


In [11]:
n_features['purpose'] = 'purpose' + n_features['purpose'].astype(str)
n_features['fueltype'] = 'fueltype' + n_features['fueltype'].astype(str)
n_features['faretype'] = 'faretype' + n_features['faretype'].astype(str)
n_features['bus_scale'] = 'bus_scale' + n_features['bus_scale'].astype(str)
n_features['travel_month'] = 'travel_month' + n_features['travel_month'].astype(str)
n_features['day_of_week'] = 'day_of_week' + n_features['day_of_week'].astype(str)
n_features['female'] = 'female' + n_features['female'].astype(str)
n_features['driving_license'] = 'driving_license' + n_features['driving_license'].astype(str)
n_features['car_ownership'] = 'car_ownership' + n_features['car_ownership'].astype(str)
n_features['pt_interchanges'] = 'pt_interchanges' + n_features['pt_interchanges'].astype(str)
n_features['age_group'] = 'age_group' + n_features['age_group'].astype(str)


n_features.head()

,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group
0,purpose1,fueltype1,faretype1,bus_scale1.0,travel_month11,day_of_week2,female0,driving_license1,car_ownership1,pt_interchanges1,age_group31-65,distance08,afternoon,high_cost,costdrivingfuelgroup8,2medium_traffic,low_charge
1,purpose3,fueltype6,faretype1,bus_scale1.0,travel_month8,day_of_week1,female1,driving_license0,car_ownership0,pt_interchanges0,age_group31-65,distance03,afternoon,low_cost,costdrivingfuelgroup4,0low_traffic,low_charge
2,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month10,day_of_week1,female1,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance09,afternoon,low_cost,costdrivingfuelgroup8,0low_traffic,low_charge
3,purpose5,fueltype1,faretype5,bus_scale0.0,travel_month1,day_of_week3,female1,driving_license1,car_ownership2,pt_interchanges0,age_group66+,distance03,afternoon,low_cost,costdrivingfuelgroup4,2medium_traffic,low_charge
4,purpose5,fueltype6,faretype1,bus_scale1.0,travel_month12,day_of_week5,female0,driving_license1,car_ownership0,pt_interchanges0,age_group31-65,distance01,afternoon,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge


combination may require for the input to be a dataframe 

In [15]:
comb_split = str(n_features.columns[1])+"*"+str(n_features.columns[2])
n_features[comb_split] = n_features[n_features.columns[1]] + n_features[n_features.columns[2]] 

n_features.head(10)

,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group,fueltype*faretype
0,purpose1,fueltype1,faretype1,bus_scale1.0,travel_month11,day_of_week2,female0,driving_license1,car_ownership1,pt_interchanges1,age_group31-65,distance08,afternoon,high_cost,costdrivingfuelgroup8,2medium_traffic,low_charge,fueltype1faretype1
1,purpose3,fueltype6,faretype1,bus_scale1.0,travel_month8,day_of_week1,female1,driving_license0,car_ownership0,pt_interchanges0,age_group31-65,distance03,afternoon,low_cost,costdrivingfuelgroup4,0low_traffic,low_charge,fueltype6faretype1
2,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month10,day_of_week1,female1,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance09,afternoon,low_cost,costdrivingfuelgroup8,0low_traffic,low_charge,fueltype1faretype1
3,purpose5,fueltype1,faretype5,bus_scale0.0,travel_month1,day_of_week3,female1,driving_license1,car_ownership2,pt_interchanges0,age_group66+,distance03,afternoon,low_cost,costdrivingfuelgroup4,2medium_traffic,low_charge,fueltype1faretype5
4,purpose5,fueltype6,faretype1,bus_scale1.0,travel_month12,day_of_week5,female0,driving_license1,car_ownership0,pt_interchanges0,age_group31-65,distance01,afternoon,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge,fueltype6faretype1
5,purpose1,fueltype1,faretype1,bus_scale1.0,travel_month11,day_of_week2,female1,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance06,early_morning,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge,fueltype1faretype1
6,purpose5,fueltype1,faretype1,bus_scale1.0,travel_month9,day_of_week4,female0,driving_license1,car_ownership1,pt_interchanges0,age_group31-65,distance01,afternoon,low_cost,costdrivingfuelgroup1,4high_traffic,high_charge,fueltype1faretype1
7,purpose3,fueltype2,faretype1,bus_scale1.0,travel_month2,day_of_week7,female1,driving_license1,car_ownership1,pt_interchanges0,age_group31-65,distance03,afternoon,low_cost,costdrivingfuelgroup2,0low_traffic,low_charge,fueltype2faretype1
8,purpose1,fueltype1,faretype1,bus_scale1.0,travel_month11,day_of_week2,female1,driving_license0,car_ownership1,pt_interchanges2,age_group31-65,distance09,evening,high_cost,costdrivingfuelgroup9,1mediumlow_traffic,low_charge,fueltype1faretype1
9,purpose3,fueltype6,faretype5,bus_scale0.0,travel_month8,day_of_week6,female0,driving_license0,car_ownership0,pt_interchanges0,age_group0-18,distance07,afternoon,low_cost,costdrivingfuelgroup7,2medium_traffic,low_charge,fueltype6faretype5


In [12]:
from cart import *

my_tree = MyNodeClass('n1', indici) 
cart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "gini",problem="classifier",method = "LATENT-BUDGET-TREE",max_level = 4, min_cases_parent= 1000,min_cases_child= 500,min_imp_gain=0.0001) 
cart.growing_tree(my_tree, max_k = 5, combination_split = True)


level 0
Var name is not among the supplied features!
TODO evaluation for lbt beyond using their errors
TypeError


combination split 

In [13]:
#to use eval, the datatype need to be str

#df = n_features
#
#y = pd.DataFrame(y)
#df["y"] = y["travel_mode"]
#
#
#df.head()


#X_test.reset_index(inplace = True, drop = True)
#y_test.reset_index(inplace = True, drop = True)
#
#X_test=X_test.applymap(str)
##y_test=y_test.applymap(str)
#
#y_test = y_test["travel_mode"] #y_test must be a series (not a dataframe)
#
#
#features_test = []
#n_features_test = X_test


#alpha = cart.pruning(features_test, n_features_test, y_test)
#cart.print_alpha(alpha) 

In [14]:

import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 


cart.print_tree(table = True, html = True).style.hide_index()



ValueError: max() arg is an empty sequence

In [ ]:
'''#run for comparison with lba 
my_tree = MyNodeClass('n1', indici) 
cart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "tau",problem="classifier",method = "TWO-STAGE",max_level = 2, min_cases_parent= 1000,min_cases_child= 500) 
cart.growing_tree(my_tree)
'''

In [ ]:
#cart.print_tree(table = False, html= False) #returns panda dataframe atm 


In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data



robjects.r("library(utils, quietly = TRUE)")
robjects.r("library(base, quietly = TRUE)")
robjects.r("suppressWarnings(install.packages('lba', quiet = TRUE))")
robjects.r("suppressMessages(library(lba, quietly = TRUE))")




In [ ]:

cont = pd.crosstab(index = n_features["distance_group"], columns= y, normalize = 'index')
#converts into an r dataframe
with (robjects.default_converter + pandas2ri.converter).context():
    cont_r = robjects.conversion.get_conversion().py2rpy(cont)

In [ ]:
robjects.r.assign("cont_r", cont_r)
robjects.r("cont_r <-  as.matrix(cont_r)")

robjects.r("out <- lba(cont_r, K = 2 , what = 'outer', method = 'ls', trace.lba = FALSE)")
print(robjects.r("out$val_func"))

robjects.r("error <- out$val_func")
robjects.r("alpha <- out$A")
robjects.r("beta <- out$B")

#out = lba.lba(base.as_matrix(cont_r), K = 2 , what = 'outer', method = 'ls')

In [ ]:
alpha  = robjects.r('alpha')
alpha=np.asarray(alpha)
alpha